# Recurrent Neural Networks and Language Models

You guys probably very excited about ChatGPT.  In today class, we will be implementing a very simple language model, which is basically what ChatGPT is, but with a simple LSTM.  You will be surprised that it is not so difficult at all.

Paper that we base on is *Regularizing and Optimizing LSTM Language Models*, https://arxiv.org/abs/1708.02182

In [11]:
# !pip install torchtext
# !pip install nepalikit
# !pip install nepalitokenizers

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext, datasets, math
from tqdm import tqdm

In [2]:
# tokenizer to tokenize nepali
from nepalitokenizers import WordPiece


In [9]:

text = "हाम्रा सबै क्रियाकलापहरु भोलिवादी छन् ।"
tokenizer = WordPiece()
tokens = tokenizer.encode(text)
print(tokens.tokens)


['[CLS]', 'हाम्रा', 'सबै', 'क्रियाकलाप', '##हरु', 'भोलि', '##वादी', 'छन्', '।', '[SEP]']


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# torch.cuda.get_device_name(0)

cuda


## 1. Load data - Wiki Text

We will be using wikitext which contains a large corpus of text, perfect for language modeling task.  This time, we will use the `datasets` library from HuggingFace to load.

In [11]:
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

#there are raw and preprocessed version; we used the raw one and preprocessed ourselves for fun
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [6]:
datasets.load_dataset?

Signature:
datasets.load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Union[str, collections.abc.Sequence[str], collections.abc.Mapping[str, Union[str, collections.abc.Sequence[str]]], NoneType] = None,
    split: Union[str, datasets.splits.Split, list[str], list[datasets.splits.Split], NoneType] = None,
    cache_dir: Optional[str] = None,
    features: Optional[datasets.features.features.Features] = None,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None,
    verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None,
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    token: Union[bool, str, NoneType] = None,
    streaming: bool = False,
    num_proc: Optional

In [6]:
dataset = datasets.load_dataset("Sakonii/nepalitext-language-model-dataset")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 13141222
    })
    test: Dataset({
        features: ['text'],
        num_rows: 268189
    })
})


In [12]:
print(dataset['train'][333]['text'])

'''
If you try to change the index you might notice that sometimes there is no paragraph 
and rather an empty string so we will have to care of that later.
'''

 During the same time frame as the Hitchcock rumors , goaltender Curtis Sanford returned from his groin injury on November 13 . He made his first start of the season against the Boston Bruins , losing 2 – 1 in a shootout . Sanford continued his strong play , posting a 3 – 1 – 2 record , 1 @.@ 38 goals against average and .947 save percentage over his next six games . Sanford started 12 consecutive games before Steve Mason made his next start . The number of starts might not have been as numerous , but prior to the November 23 game , Mason was hit in the head by a shot from Rick Nash during pre @-@ game warm @-@ ups and suffered a concussion . Mason returned from his concussion after two games , making a start against the Vancouver Canucks . Mason allowed only one goal in the game despite suffering from cramping in the third period , temporarily being replaced by Sanford for just over three minutes . Columbus won the game 2 – 1 in a shootout , breaking a nine @-@ game losing streak to t

'\nIf you try to change the index you might notice that sometimes there is no paragraph \nand rather an empty string so we will have to care of that later.\n'

## 2. Preprocessing

### Tokenizing

Simply tokenize the given text to tokens.

In [ ]:
# use WordPiece tokenizer for Nepali
tokenizer = WordPiece()

def tokenize_data(example, tokenizer):
    text = example.get('text', '') or ''
    if not text:
        return {'tokens': []}
    enc = tokenizer.encode(text)
    toks = enc.tokens if hasattr(enc, 'tokens') else list(enc)
    return {'tokens': toks}

# map the function to each example
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
print(tokenized_dataset['train'][333]['tokens'])

Map:   0%|          | 0/13141222 [00:00<?, ? examples/s]

KeyboardInterrupt: 

### Numericalizing

We will tell torchtext to add any word that has occurred at least three times in the dataset to the vocabulary because otherwise it would be too big.  Also we shall make sure to add `unk` and `eos`.

In [ ]:
## numericalizing (build vocab from ~50k tokens)
# collect up to 50k tokens from train split
max_tokens = 50000
collected = []
for ex in tokenized_dataset['train']:
    toks = ex['tokens']
    if toks:
        collected.extend(toks + ['<eos>'])
    if len(collected) >= max_tokens:
        collected = collected[:max_tokens]
        break

def iterator():
    yield collected

vocab = torchtext.vocab.build_vocab_from_iterator(iterator(), min_freq=3, specials=['<unk>', '<eos>'])
vocab.set_default_index(vocab['<unk>'])
print('Collected tokens for vocab:', len(collected))
print('Vocab size:', len(vocab))
print(vocab.get_itos()[:10])

TypeError: build_vocab_from_iterator() got an unexpected keyword argument 'min_freq'

## 3. Prepare the batch loader

### Prepare data

Given "Chaky loves eating at AIT", and "I really love deep learning", and given batch size = 3, we will get three batches of data "Chaky loves eating at", "AIT `<eos>` I really", "love deep learning `<eos>`".  

In [ ]:
def tokens_list_to_data(tokens_list, vocab, batch_size):
    ids = [vocab[token] if token in vocab.get_itos() else vocab['<unk>'] for token in tokens_list]
    data = torch.LongTensor(ids)
    num_batches = data.shape[0] // batch_size
    data = data[: num_batches * batch_size]
    data = data.view(batch_size, -1)
    return data

In [ ]:
# create train/valid/test splits from collected tokens (90/5/5)
batch_size = 128
n = len(collected)
train_tokens = collected[: int(n * 0.9)]
valid_tokens = collected[int(n * 0.9): int(n * 0.95)]
test_tokens  = collected[int(n * 0.95):]

train_data = tokens_list_to_data(train_tokens, vocab, batch_size)
valid_data = tokens_list_to_data(valid_tokens, vocab, batch_size) if len(valid_tokens) >= batch_size else train_data
test_data  = tokens_list_to_data(test_tokens, vocab, batch_size) if len(test_tokens) >= batch_size else train_data

print('train_data.shape', train_data.shape)

In [10]:
train_data.shape #[batch size, all the text length]

torch.Size([128, 16214])

## 4. Modeling 

In [12]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers, dropout_rate):
                
        super().__init__()
        self.hid_dim   = hid_dim
        self.num_layers= num_layers
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm      = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers,
                                 dropout=dropout_rate, batch_first=True)
        self.dropout   = nn.Dropout(dropout_rate)
        #when you do LM, you look forward, so it does not make sense to do bidirectionality
        self.fc        = nn.Linear(hid_dim, vocab_size)

    def init_hidden(self, batch_size, device):
        #this function gonna be run in the beginning of the epoch
        hidden = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        cell   = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        
        return hidden, cell #return as tuple
    
    def detach_hidden(self, hidden):
        #this gonna run in every batch
        hidden, cell = hidden
        hidden = hidden.detach() #removing this hidden from gradients graph
        cell   = cell.detach()   #removing this cell from gradients graph
        return hidden, cell
        
    def forward(self, src, hidden):
        #src: [batch size, seq len]
        
        #embed
        embed = self.embedding(src)
        #embed: [batch size, seq len, emb_dim]
        
        #send this to the lstm
        #we want to put hidden here...because we want to reset hidden....
        output, hidden = self.lstm(embed, hidden)
        #output: [batch size, seq len, hid_dim] ==> all hidden states
        #hidden: [num layer, batch size, hid_dim]  ===> last hidden states from each layer
        
        output = self.dropout(output)
        prediction = self.fc(output)
        #prediction: [batch size, seq len, vocab size]
        
        return prediction, hidden
    

## 5. Training 

Follows very basic procedure.  One note is that some of the sequences that will be fed to the model may involve parts from different sequences in the original dataset or be a subset of one (depending on the decoding length). For this reason we will reset the hidden state every epoch, this is like assuming that the next batch of sequences is probably always a follow up on the previous in the original dataset.

In [13]:
vocab_size = len(vocab)
emb_dim = 1024                # 400 in the paper
hid_dim = 1024                # 1150 in the paper
num_layers = 2                # 3 in the paper
dropout_rate = 0.65              
lr = 1e-3                     

In [14]:
model = LSTMLanguageModel(vocab_size, emb_dim, hid_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 77,183,777 trainable parameters


In [15]:
def get_batch(data, seq_len, idx):
    #this data is from get_data()
    #train_data.shape #[batch size, number of batches....]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [16]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  #prevents gradient explosion - clip is basically the threshold.....
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [17]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [18]:
n_epochs = 50
seq_len  = 50
clip     = 0.25

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

best_valid_loss = float('inf')

for epoch in range(n_epochs):
    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)

    lr_scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-lstm_lm.pt')

    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

KeyboardInterrupt: 

## 6. Testing

In [ ]:
model.load_state_dict(torch.load('best-val-lstm_lm.pt',  map_location=device))
test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
print(f'Test Perplexity: {math.exp(test_loss):.3f}')

## 7. Real-world inference

Here we take the prompt, tokenize, encode and feed it into the model to get the predictions.  We then apply softmax while specifying that we want the output due to the last word in the sequence which represents the prediction for the next word.  We divide the logits by a temperature value to alter the model’s confidence by adjusting the softmax probability distribution.

Once we have the Softmax distribution, we randomly sample it to make our prediction on the next word. If we get <unk> then we give that another try.  Once we get <eos> we stop predicting.
    
We decode the prediction back to strings last lines.

In [ ]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens